In [32]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [33]:
#importing the libraries
import pandas as pd
from glob import glob
import numpy as np
import pywt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from google.cloud import storage
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc, roc_auc_score
import time

In [149]:
#funtion to load data train

def prepare_data_train(fname):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(fname)
    # events file
    events_fname = fname.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels

In [90]:
#function to import one subject
def loading_one_subject(subject_number):
    y_raw= []
    raw = []
    fnames =  sorted(glob('/content/drive/MyDrive/data/hand_motion/train/subj%d_series*_data.csv' % (subject_number)))
    for fname in fnames:
        data,labels=prepare_data_train(fname)
        raw.append(data)
        y_raw.append(labels)
    X = pd.concat(raw).reset_index()
    X.drop(columns="index", inplace=True)
    y = pd.concat(y_raw).reset_index()
    y.drop(columns="index", inplace=True)
    return X, y

In [150]:
#importing subject 1
X, y = loading_one_subject(1) 

## Function for mean absolute deviation of a signal and wavelet_denoising

In [138]:
def convert_df_to_numpy(df):
  return np.asarray(df.astype(float))
  

In [95]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [137]:
def wavelet_denoising(X, wavelet='db2', level=3):
    coeff = pywt.wavedec(X, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(X)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


In [97]:
scaler= StandardScaler()
def data_scaler_train(X):
    X_prep=scaler.fit_transform(X)
    #do here your preprocessing
    return X_prep
def data_scaler_test(X):
    X_prep=scaler.transform(X)
    #do here your preprocessing
    return X_prep

In [151]:
def preprocess_data(X, y):
  X= convert_df_to_numpy(X)
  y=convert_df_to_numpy(y)
  X=wavelet_denoising(X)
  return X, y

In [ ]:
x_train=data_scaler_train(x_train_butter)

In [141]:
x_train_1, y_1 = preprocess_data(X, y)

**bold text**## Splitting data

In [98]:
def custom_train_test_split(X, y):
     
    splitrate=-x_train.shape[0]//5*2
    xval=x_train[splitrate:splitrate//2]
    yval=y[splitrate:splitrate//2]
    xtest=x_train[splitrate//2:]
    ytest=y[splitrate//2:]
    xtrain=x_train[:splitrate]
    ytrain=y[:splitrate]
    
    return xtrain, xval, xtest, ytrain, yval, ytest 

In [99]:
# splitting the data 
xtrain, xval, xtest, ytrain, yval, ytest = custom_train_test_split(X, y)

In [51]:
xtrain.shape, xval.shape, xtest.shape, ytrain.shape, yval.shape, ytest.shape

((853434, 32),
 (284479, 32),
 (284479, 32),
 (853434, 6),
 (284479, 6),
 (284479, 6))

## Generators


In [100]:
time_steps = 1000
subsample = 50

In [101]:
def generator(batch_size, xtrain, ytrain):
    ## Is xtrain passed onto the function as a parameter
    time_steps = 1000
    subsample = 50
    while 1:
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtrain)-time_steps)
            x_time_data[i] = xtrain[random_index:random_index+time_steps:subsample]
            yy.append(ytrain[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1],  x_time_data.shape[2],1)), yy

In [102]:
def valgenerator(xval, yval):
    time_steps = 1000
    subsample = 50
    while 1:
        batch_size=32
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xval)-time_steps)
            x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
            yy.append(yval[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy

In [104]:
for i in generator(32, xtrain, ytrain):
  type(i)

KeyError: ignored

In [80]:
xval.shape

(284479, 32)

In [85]:
yval.shape

(284479, 6)

In [108]:
yval.iloc[81017]

HandStart             0
FirstDigitTouch       0
BothStartLoadPhase    0
LiftOff               0
Replace               0
BothReleased          0
Name: 934451, dtype: int64

In [110]:
type(xval)

numpy.ndarray

In [111]:
type(yval)

pandas.core.frame.DataFrame

In [109]:

batch_size=32
x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
yy = []
alist=[]
for i in range(batch_size):
    random_index = np.random.randint(0, len(xval)-time_steps)
    alist.append(random_index)
    x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
    yy.append(yval.iloc[random_index + time_steps])
#yy = np.asarray(yy)
#yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy

AttributeError: ignored

In [75]:
max(alist)

264792

In [58]:
x_time_data.shape

(32, 20, 32)

## Defining one Model

In [48]:
load = 1
time_steps = 1000
subsample = 50

model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = ((time_steps//subsample), 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(6, activation = "sigmoid"))


adam = Adam(lr = 0.0001)

model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ['accuracy','mse'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 20, 32, 64)        3200      
_________________________________________________________________
batch_normalization_8 (Batch (None, 20, 32, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 32, 64)        102464    
_________________________________________________________________
batch_normalization_9 (Batch (None, 20, 32, 64)        256       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 32, 64)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 20, 32, 64)        256       
_________________________________________________________________
flatten_2 (Flatten)          (None, 40960)            

In [49]:
generator_ = generator(32, xtrain, ytrain)
valgenerator_ = valgenerator(xval, yval)


In [68]:
generator_

<generator object generator at 0x7f37ceaa3e50>

In [69]:
for i in generator_:
  print(type (i))

In [59]:
es= EarlyStopping(patience=2)
history =model.fit_generator(generator_, steps_per_epoch = 600, epochs = 50,validation_data=valgenerator_,
                              validation_steps=200,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


KeyError: ignored

In [ ]:
es= EarlyStopping(patience=2)
history =model.fit_generator(generator(32, xtrain, ytrain), steps_per_epoch = 600, epochs = 50,validation_data=valgenerator(xval, yval),
                              validation_steps=200,callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


KeyError: ignored

In [ ]:
X_train =np.asarray(X.astype(float))
y_train = np.asarray(y.astype(float))

In [ ]:
def valgenerator():
    while 1:
        batch_size=32
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        print(x_time_data)
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xval)-time_steps)
            x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
            yy.append(yval[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping
start=time.time()
def generator(batch_size):
    while 1:
        
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtrain)-time_steps)
            x_time_data[i] = xtrain[random_index:random_index+time_steps:subsample]
            yy.append(ytrain[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1],  x_time_data.shape[2],1)), yy
        
es= EarlyStopping(patience=2)
history =model.fit_generator(generator(32), steps_per_epoch = 600, epochs = 50,validation_data=valgenerator(),
                              validation_steps=200,callbacks=[es])
#print('training time taken: ',round(time.time()-start,0),'seconds')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


KeyError: ignored